# Autoscaling TensorFlow Model Deployments with TF Serving and Kubernetes

In [ ]:
gcloud auth list
gcloud config list project

### Task 1. Getting lab files

In [ ]:
cd
SRC_REPO=https://github.com/GoogleCloudPlatform/mlops-on-gcp
kpt pkg get $SRC_REPO/workshops/mlep-qwiklabs/tfserving-gke-autoscaling tfserving-gke
cd tfserving-gke

### Task 2. Creating a GKE cluster

In [ ]:
gcloud config set compute/zone us-central1-f
PROJECT_ID=$(gcloud config get-value project)
CLUSTER_NAME=cluster-1

gcloud beta container clusters create $CLUSTER_NAME \
    --cluster-version=latest \
    --machine-type=n1-standard-4 \
    --enable-autoscaling \
    --min-nodes=1 \
    --max-nodes=3 \
    --num-nodes=1

gcloud container clusters get-credentials $CLUSTER_NAME 

### Task 3. Deploying ResNet101

In [ ]:
export MODEL_BUCKET=${PROJECT_ID}-bucket
gsutil mb gs://${MODEL_BUCKET}

gsutil cp -r gs://spls/gsp777/resnet_101 gs://${MODEL_BUCKET}

### Task 4. Creating ConfigMap

In [ ]:
echo $MODEL_BUCKET

kubectl apply -f tf-serving/configmap.yaml

### Task 5. Creating TensorFlow Serving deployment

In [ ]:
cat tf-serving/deployment.yaml

kubectl apply -f tf-serving/deployment.yaml

kubectl get deployments

### Task 6. Exposing the deployment

In [ ]:
cat tf-serving/service.yaml

kubectl apply -f tf-serving/service.yaml

kubectl get svc image-classifier

### Task 7. Configuring horizontal pod autoscaler

In [ ]:
kubectl autoscale deployment image-classifier \
    --cpu-percent=60 \
    --min=1 \
    --max=4

kubectl get hpa

### Task 8. Testing the model

In [ ]:
EXTERNAL_IP=34.70.145.13
curl -d @locust/request-body.json \
    -X POST http://${EXTERNAL_IP}:8501/v1/models/image_classifier:predict

### Task 9. Installing Locust

In [ ]:
pip3 install locust==1.4.1

export PATH=~/.local/bin:$PATH

locust -V

### Task 10. Starting a load test

In [ ]:
cd locust
locust -f tasks.py \
    --headless \
    --host http://${EXTERNAL_IP}:8501

### Task 11. Monitoring the load test

In [ ]:
https://console.cloud.google.com/kubernetes/deployment/us-central1-f/cluster-1/default/image-classifier/overview

https://console.cloud.google.com/kubernetes/nodepool/us-central1-f/cluster-1/default-pool